**This is example of how to trace model with jit and export it to the onnx**

In [1]:
from rl_games.torch_runner import Runner
import ray
import yaml
import torch
import matplotlib.pyplot as plt
import gym
from IPython import display
import numpy as np
import onnx
import onnxruntime as ort
%matplotlib inline

/home/viktorm/anaconda3/envs/warp39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/viktorm/anaconda3/envs/warp39/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, '__version__') or LooseVersion(tensorboard.__version__) < LooseVersion('1.15'):


In [2]:
ray.init(object_store_memory=1024*1024*1000)

/home/viktorm/anaconda3/envs/warp39/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


RayContext(dashboard_url='', python_version='3.9.12', ray_version='1.12.0', ray_commit='f18fc31c7562990955556899090f8e8656b48d2d', address_info={'node_ip_address': '10.2.168.57', 'raylet_ip_address': '10.2.168.57', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-09-16_13-07-38_643208_1200015/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-09-16_13-07-38_643208_1200015/sockets/raylet', 'webui_url': '', 'session_dir': '/tmp/ray/session_2022-09-16_13-07-38_643208_1200015', 'metrics_export_port': 58850, 'gcs_address': '10.2.168.57:47337', 'address': '10.2.168.57:47337', 'node_id': 'a41e8ed5dcebcaabbb00fcc7b042adca6e16353a1c7cd5fe7272e7a3'})

In [3]:
config_name = '../rl_games/configs/ppo_pendulum_torch.yaml'

In [4]:
with open(config_name, 'r') as stream:
    config = yaml.safe_load(stream)
    config['params']['config']['normalize_input'] = True
    config['params']['config']['max_epochs'] = 200
    config['params']['config']['full_experiment_name'] = 'pendulum_onnx'
runner = Runner()
runner.load(config)
runner.run({
    'train': True,
})

self.seed = 1663358860
Started to train
Exact experiment name requested from command line: pendulum_onnx


(RayWorker pid=1200387) /home/viktorm/anaconda3/envs/warp39/lib/python3.9/site-packages/gym/core.py:172: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed) instead.
(RayWorker pid=1200387)   deprecation(
(RayWorker pid=1200383) /home/viktorm/anaconda3/envs/warp39/lib/python3.9/site-packages/gym/core.py:172: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed) instead.
(RayWorker pid=1200383)   deprecation(
(RayWorker pid=1200388) /home/viktorm/anaconda3/envs/warp39/lib/python3.9/site-packages/gym/core.py:172: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed) instead.
(RayWorker pid=1200388)   deprecation(
(RayWorker pid=1200389) /home/viktorm/anaconda3/envs/warp39/lib/python3.9/site-packages/gym/core.py:172: Depr

current training device: cuda:0
build mlp: 3
build mlp: 3
RunningMeanStd:  (3,)
fps step: 2401 fps step and policy inference: 1051 fps total: 1028 epoch: 1/200
fps step: 2456 fps step and policy inference: 1879 fps total: 1799 epoch: 2/200
fps step: 2571 fps step and policy inference: 1950 fps total: 1893 epoch: 3/200
fps step: 2325 fps step and policy inference: 1781 fps total: 1702 epoch: 4/200
fps step: 2388 fps step and policy inference: 1814 fps total: 1756 epoch: 5/200
fps step: 2507 fps step and policy inference: 1899 fps total: 1820 epoch: 6/200
fps step: 2603 fps step and policy inference: 1983 fps total: 1923 epoch: 7/200
fps step: 2594 fps step and policy inference: 1951 fps total: 1872 epoch: 8/200
fps step: 2589 fps step and policy inference: 1968 fps total: 1903 epoch: 9/200
fps step: 2595 fps step and policy inference: 1962 fps total: 1879 epoch: 10/200
fps step: 2635 fps step and policy inference: 1989 fps total: 1927 epoch: 11/200
fps step: 2641 fps step and policy inf

fps step: 3321 fps step and policy inference: 2557 fps total: 2460 epoch: 101/200
fps step: 3278 fps step and policy inference: 2522 fps total: 2439 epoch: 102/200
fps step: 3325 fps step and policy inference: 2550 fps total: 2463 epoch: 103/200
fps step: 3305 fps step and policy inference: 2532 fps total: 2439 epoch: 104/200
fps step: 3345 fps step and policy inference: 2555 fps total: 2469 epoch: 105/200
fps step: 3314 fps step and policy inference: 2534 fps total: 2447 epoch: 106/200
fps step: 3338 fps step and policy inference: 2561 fps total: 2475 epoch: 107/200
fps step: 3339 fps step and policy inference: 2572 fps total: 2484 epoch: 108/200
fps step: 3335 fps step and policy inference: 2559 fps total: 2471 epoch: 109/200
fps step: 3383 fps step and policy inference: 2591 fps total: 2499 epoch: 110/200
fps step: 3354 fps step and policy inference: 2580 fps total: 2486 epoch: 111/200
fps step: 3368 fps step and policy inference: 2603 fps total: 2501 epoch: 112/200
fps step: 3223 f

In [8]:
class ModelWrapper(torch.nn.Module):
    '''
    Main idea is to ignore outputs which we don't need from model
    '''
    def __init__(self, model):
        torch.nn.Module.__init__(self)
        self._model = model
        
        
    def forward(self,input_dict):
        input_dict['obs'] = self._model.norm_obs(input_dict['obs'])
        '''
        just model export doesn't work. Looks like onnx issue with torch distributions
        thats why we are exporting only neural network
        '''
        #print(input_dict)
        #output_dict = self._model.a2c_network(input_dict)
        #input_dict['is_train'] = False
        #return output_dict['logits'], output_dict['values']
        return self._model.a2c_network(input_dict)

In [9]:
agent = runner.create_player()
agent.restore('runs/pendulum_onnx/nn/pendulum.pth')

import rl_games.algos_torch.flatten as flatten
inputs = {
    'obs' : torch.zeros((1,) + agent.obs_shape).to(agent.device),
    'rnn_states' : agent.states,
}

with torch.no_grad():
    adapter = flatten.TracingAdapter(ModelWrapper(agent.model), inputs,allow_non_tensor=True)
    traced = torch.jit.trace(adapter, adapter.flattened_inputs,check_trace=False)
    flattened_outputs = traced(*adapter.flattened_inputs)
    print(flattened_outputs)
    
torch.onnx.export(traced, *adapter.flattened_inputs, "pendulum.onnx", verbose=True, input_names=['obs'], output_names=['mu','log_std', 'value'])

onnx_model = onnx.load("pendulum.onnx")

# Check that the model is well formed
onnx.checker.check_model(onnx_model)

{'observation_space': Box([-1. -1. -8.], [1. 1. 8.], (3,), float32), 'action_space': Box(-2.0, 2.0, (1,), float32), 'agents': 1, 'value_size': 1}
build mlp: 3
build mlp: 3
RunningMeanStd:  (3,)
=> loading checkpoint 'runs/pendulum_onnx/nn/pendulum.pth'
1 1
(tensor([[0., 0., 0.]], device='cuda:0'),) (tensor([[0., 0., 0.]], device='cuda:0'),)
(tensor([[0.4290]], device='cuda:0'), tensor([[-1.4877]], device='cuda:0'), tensor([[-0.9674]], device='cuda:0'))


/home/viktorm/anaconda3/envs/warp39/lib/python3.9/site-packages/torch/onnx/utils.py:359: UserWarning: Model has no forward function
  warnings.warn("Model has no forward function")


graph(%obs : Float(1, 3, strides=[3, 1], requires_grad=0, device=cuda:0),
      %model._model.a2c_network.sigma : Float(1, strides=[1], requires_grad=0, device=cuda:0),
      %model._model.a2c_network.actor_mlp.0.bias : Float(32, strides=[1], requires_grad=0, device=cuda:0),
      %model._model.a2c_network.actor_mlp.0.weight : Float(32, 3, strides=[3, 1], requires_grad=0, device=cuda:0),
      %model._model.a2c_network.actor_mlp.2.bias : Float(32, strides=[1], requires_grad=0, device=cuda:0),
      %model._model.a2c_network.actor_mlp.2.weight : Float(32, 32, strides=[32, 1], requires_grad=0, device=cuda:0),
      %model._model.a2c_network.critic_mlp.0.bias : Float(32, strides=[1], requires_grad=0, device=cuda:0),
      %model._model.a2c_network.critic_mlp.0.weight : Float(32, 3, strides=[3, 1], requires_grad=0, device=cuda:0),
      %model._model.a2c_network.critic_mlp.2.bias : Float(32, strides=[1], requires_grad=0, device=cuda:0),
      %model._model.a2c_network.critic_mlp.2.weight :

In [11]:
ort_model = ort.InferenceSession("pendulum.onnx")

outputs = ort_model.run(
    None,
    {"obs": np.zeros((1, 3)).astype(np.float32)},
)
print(outputs)

[array([[0.42947608]], dtype=float32), array([[-1.4877307]], dtype=float32), array([[-0.9669779]], dtype=float32)]


In [15]:
is_done = False
env = agent.env
obs = env.reset()
#prev_screen = env.render(mode='rgb_array')
#plt.imshow(prev_screen)
total_reward = 0
num_steps = 0
while not is_done:
    outputs = ort_model.run(None, {"obs": np.expand_dims(obs, axis=0).astype(np.float32)},)
    action = outputs[0].squeeze(1)
    print(action)
    obs, reward, done, info = env.step(action)
    total_reward += reward
    num_steps += 1
    is_done = done
    screen = env.render(mode='rgb_array')
    #plt.imshow(screen)
    #display.display(plt.gcf())    
    #display.clear_output(wait=True)
print(total_reward, num_steps)
#ipythondisplay.clear_output(wait=True)

[2.5744312]
[3.0435042]
[3.3461144]
[3.5552688]
[3.6558933]
[3.6372302]
[3.5011947]
[3.2604012]
[2.9250076]
[2.5060184]
[2.0745935]
[1.6998118]
[1.5270209]
[1.4712086]
[1.2709275]
[0.9568208]
[0.6169096]
[0.29001135]
[-0.03854244]
[-0.3977334]
[-0.7772378]
[-1.119332]
[-1.3981546]
[-1.5736771]
[-1.6208159]
[-1.5204344]
[-1.2796079]
[-0.9408687]
[-0.573925]
[-0.3071589]
[-0.2773655]
[-0.42557085]
[-0.57951653]
[-0.678273]
[-0.7442626]
[-0.79648864]
[-0.84287536]
[-0.90002215]
[-0.9714671]
[-1.0369996]
[-1.0965557]
[-1.1487722]
[-1.1943798]
[-1.2349553]
[-1.2719566]
[-1.3064971]
[-1.3392601]
[-1.370506]
[-1.4000406]
[-1.4271226]
[-1.4528034]
[-1.4788148]
[-1.5048565]
[-1.5302626]
[-1.5538046]
[-1.5733638]
[-1.5853615]
[-1.5838745]
[-1.5625072]
[-1.5170943]
[-1.4410225]
[-1.3232167]
[-1.149319]
[-0.9054007]
[-0.5798164]
[-0.15795323]
[0.36813432]
[0.99135995]
[1.6637182]
[2.2105649]
[2.4795938]
[2.4223804]
[2.0225987]
[1.3603784]
[0.6703036]
[0.2163481]
[-0.11333267]
[-0.39264995]
[-0.660